In [17]:
import json
import psycopg2
from datetime import date

# Function to connect to the PostgreSQL database
def connect_to_db():
    try:
        # Replace these values with your connection details
        conn = psycopg2.connect(
            dbname='postgres', 
            user='postgres', 
            password='jhyu1982.1', 
            host='prepopulation.c9k0s64g0471.eu-north-1.rds.amazonaws.com', 
            port='5432'
        )
        return conn
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None

# Function to insert an artist into the database
def insert_artist(conn, artist):
    with conn.cursor() as cur:
        third_image_url = artist['images'][2]['url'] if len(artist['images']) >= 3 else None
        sql = """
        INSERT INTO mainartist (dateAddedToDB, dateLastModified, artistSpotifyID, artistName, artistPopularity, artistImage, artistFollowers)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (artistSpotifyID) DO NOTHING;
        """
        cur.execute(sql, (date.today(), date.today(), artist['id'], artist['name'], artist['popularity'], third_image_url, artist['followers']['total']))
        conn.commit()

# Main function to process the JSON file and insert data into the database
def process_json_file(filepath):
    conn = connect_to_db()
    if conn is not None:
        with open(filepath, 'r') as file:
            data = json.load(file)
            for key in data.keys():
                for artist in data[key]:
                    insert_artist(conn, artist)
        conn.close()
    else:
        print("Failed to connect to the database.")

# Example usage
# process_json_file('path_to_your_json_file.json')

In [19]:
process_json_file('96kArtistInfo.json')